In [1]:
import pandas as pd
import numpy as np
import argparse
import csv
import json
import os
import random
import sys

In [2]:
cleandataset = pd.read_csv('data/dataset.csv')
cleandataset.columns

Index(['merchant_name', 'cleanName2', 'start_idx', 'end_idx'], dtype='object')

In [3]:
from sklearn.utils import shuffle
from random import shuffle

shuffle_1 = cleandataset.copy()
shuffle_2 = cleandataset.copy()

from nltk.tokenize import word_tokenize
# shuffle_1['merchant_name'] = shuffle_1['merchant_name'].apply(lambda sentence: ' '.join(random.sample(sentence.split(' '), k = len(sentence.split(' ')))))
# # shuffle_1.head()
# shuffle_2['merchant_name'] = shuffle_2['merchant_name'].apply(lambda sentence: ' '.join(random.sample(sentence.split(' '), k = len(sentence.split(' ')))))
# shuffle_2.sample(5)
for idx, row in shuffle_1.iterrows():
    if len(row[1].split()) > 1:
        try:
            sidx = row[0].find(row[1])
            eidx = sidx + len(row[1])
            merchant = row[1]
            tempsentence = row[0][:]
            insertion_pos = tempsentence.split().index(merchant.split()[0])
            tempsentence_removed_tokenized = tempsentence.replace(row[1], ' ').split()
            tempsentence_removed_tokenized.insert(insertion_pos, merchant)
            shuffled_sentence = ' '.join(random.sample(tempsentence_removed_tokenized, k = len(tempsentence_removed_tokenized)))
            shuffle_1.iloc[idx, 0] = shuffled_sentence

        except:
            continue
    else:
        shuffle_1.iloc[idx, 0] = ' '.join(random.sample(row[0].split(' '), k = len(row[0].split(' '))))

shuffle_1.reset_index(drop = True)
for idx, row in shuffle_1.iterrows():
    shuffle_1.iloc[idx, 2] = shuffle_1.iloc[idx, 0].find(shuffle_1.iloc[idx, 1])
    shuffle_1.iloc[idx, 3] = shuffle_1.iloc[idx, 2] + len(shuffle_1.iloc[idx, 1])
shuffle_1.sample(5)



,merchant_name,cleanName2,start_idx,end_idx
192022,fu shelby tn 1230 kroger memphis,kroger,18,24
182031,u 991597806779 hulu hlu,hulu,15,19
220663,susansmith7 ebay paypal,ebay,12,16
10515,0y82133n3 amazon amzn,amazon,10,16
140890,parris b dollar general,dollar general,9,23


In [4]:
for idx, row in shuffle_2.iterrows():
    if len(row[1].split()) > 1:
        try:
            sidx = row[0].find(row[1])
            eidx = sidx + len(row[1])
            merchant = row[1]
            tempsentence = row[0][:]
            insertion_pos = tempsentence.split().index(merchant.split()[0])
            tempsentence_removed_tokenized = tempsentence.replace(row[1], ' ').split()
            tempsentence_removed_tokenized.insert(insertion_pos, merchant)
            shuffled_sentence = ' '.join(random.sample(tempsentence_removed_tokenized, k = len(tempsentence_removed_tokenized)))
            shuffle_2.iloc[idx, 0] = shuffled_sentence

        except:
            continue
    else:
        shuffle_2.iloc[idx, 0] = ' '.join(random.sample(row[0].split(' '), k = len(row[0].split(' '))))

shuffle_2.reset_index(drop = True)
for idx, row in shuffle_2.iterrows():
    shuffle_2.iloc[idx, 2] = shuffle_2.iloc[idx, 0].find(shuffle_2.iloc[idx, 1])
    shuffle_2.iloc[idx, 3] = shuffle_2.iloc[idx, 2] + len(shuffle_2.iloc[idx, 1])
shuffle_2.sample(5)

,merchant_name,cleanName2,start_idx,end_idx
229208,garden fe449 postmates,postmates,13,22
79113,ys55r6hb amazon,amazon,9,15
189931,030 washing 5312 kohls,kohls,17,22
175016,u hulu hlu 1560226836583,hulu,2,6
81910,chick cincinnati aramark oh usa fil,aramark,17,24


In [8]:
result_temp = pd.concat([cleandataset, shuffle_1], axis = 0)
result = pd.concat([result_temp, shuffle_2], axis = 0)
result.sample(5)

,merchant_name,cleanName2,start_idx,end_idx
262746,parker co target usa,target,10,16
280977,r wegmans ridge,wegmans,2,9
147019,doordash slim chickens,doordash,0,8
108390,jen cash app melvin,cash app,4,12
122411,gas sunny citgo,citgo,10,15


In [12]:
result.reset_index(drop=True)

,merchant_name,cleanName2,start_idx,end_idx
0,0 04 amc creve coeur,amc,5,8
1,0 05 amc security sq,amc,5,8
2,0 1 forever 21,forever 21,4,14
3,0 2 forever 21,forever 21,4,14
4,0 3 amc grand island,amc,4,7
...,...,...,...,...
863056,bp zw100082 tn w100082 jackson,bp,0,2
863057,new zxbuzamxgxnwnst york city membership n pel...,peloton,43,50
863058,llc zyia active,zyia active,4,15
863059,zyia active llc usa 4353835263 ut,zyia active,0,11


In [13]:
result.to_csv('data/shuffled_dataset.csv')

In [20]:
# len(result) /
863060*0.07


60414.200000000004

In [75]:
TRAIN_DATA = result.values.tolist()
TRAIN_DATA = [(item[0], {'merchant': item[1], 'entities':[(item[2], item[3], 'BRD')]}) for item in TRAIN_DATA]
TRAIN_DATA[:5]

[('0 04 amc creve coeur', {'merchant': 'amc', 'entities': [(5, 8, 'BRD')]}),
 ('0 05 amc security sq', {'merchant': 'amc', 'entities': [(5, 8, 'BRD')]}),
 ('0 1 forever 21', {'merchant': 'forever 21', 'entities': [(4, 14, 'BRD')]}),
 ('0 2 forever 21', {'merchant': 'forever 21', 'entities': [(4, 14, 'BRD')]}),
 ('0 3 amc grand island', {'merchant': 'amc', 'entities': [(4, 7, 'BRD')]})]

In [21]:
import spacy
import random
from random import sample
from sklearn.model_selection import train_test_split
import warnings

warnings.filterwarnings("ignore")

def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
# add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])
# get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Starting iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

random.shuffle(TRAIN_DATA)

train = TRAIN_DATA[:int(len(TRAIN_DATA) *0.07)]
test = TRAIN_DATA[int(len(TRAIN_DATA) *0.07):]

prdnlp = train_spacy(train, 20)
# Save our trained Model
modelfile = 'wtf'
prdnlp.to_disk(modelfile)


Starting iteration 0
{'ner': 14785.280735623952}
Starting iteration 1
{'ner': 8050.384688175385}
Starting iteration 2
{'ner': 7032.637879721816}
Starting iteration 3
{'ner': 6820.436508659606}
Starting iteration 4
{'ner': 6619.630067326758}
Starting iteration 5
{'ner': 6850.296819111672}
Starting iteration 6
{'ner': 6620.3372691328905}
Starting iteration 7
{'ner': 6467.140071386397}
Starting iteration 8
{'ner': 6485.079976149564}
Starting iteration 9
{'ner': 6599.898024007217}
Starting iteration 10
{'ner': 6824.941364560558}
Starting iteration 11
{'ner': 6730.289790247451}
Starting iteration 12
{'ner': 6704.052421643572}
Starting iteration 13
{'ner': 6599.488980906231}
Starting iteration 14
{'ner': 6556.489141455509}
Starting iteration 15
{'ner': 6469.259250730547}
Starting iteration 16
{'ner': 6464.355498739713}
Starting iteration 17
{'ner': 6448.337810457536}
Starting iteration 18
{'ner': 6697.459910993406}
Starting iteration 19
{'ner': 6531.123988932317}


In [78]:
test = TRAIN_DATA[int(len(TRAIN_DATA) *0.07):]

In [90]:
#Test your text
from fuzzywuzzy import fuzz
test_text = sample(test, 1000)
scores = []
for text in test_text:
    #print('test case: ', text)
    doc = prdnlp(text[0])
    for ent in doc.ents:
        # print(ent.text)
        # print('True label is: ')
        # print(text[1]['merchant'])
        similarity = fuzz.token_sort_ratio(ent.text,text[1]['merchant'])
        #print('similarity is: ', similarity)
        scores.append(similarity)
print(scores.count(100) / len(scores) *100, ' percent of test data has been correctly classified.')


95.1  percent of test data has been correctly classified.


0.99

In [74]:
load_nlp = spacy.load('wtf')
test_text = sample(test, 10)
for text in test_text:
    print('test case: ', text)
    doc = load_nlp(text[0])
    for ent in doc.ents:
        print(ent.text)

test case:  ('paypal claudetebah ebay', {'entities': [(19, 23, 'BRD')]})
ebay
test case:  ('healthful essence', {'entities': [(0, 17, 'BRD')]})
healthful essence
test case:  ('taco bell ca murrieta qps', {'entities': [(0, 9, 'BRD')]})
taco bell
test case:  ('amazon amzn 764lv36i3', {'entities': [(0, 6, 'BRD')]})
amazon
test case:  ('pla dollar general', {'entities': [(4, 18, 'BRD')]})
dollar general
test case:  ('amazon dk5h92da3 seattle wa usa', {'entities': [(0, 6, 'BRD')]})
amazon
test case:  ('taqueria red 3 hermanos', {'entities': [(13, 23, 'BRD')]})
red 3
test case:  ('rj64g2qr amazon', {'entities': [(9, 15, 'BRD')]})
amazon
test case:  ('citgo greenf', {'entities': [(0, 5, 'BRD')]})
citgo
test case:  ('amazon c43nw12l', {'entities': [(0, 6, 'BRD')]})
amazon


In [ ]:
tempdata = pd.read_csv('data/cleanedmerchant_training.csv') ## read the file into to a pandas dataframe
data = tempdata[['merchant_name', 'cleanName2']]
validName = data[data['cleanName2'].notnull()]
validName.reset_index(inplace=True, drop = True)
validName.head(50)

In [ ]:
from nltk import word_tokenize

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# mu, sigma = 100, 10 # mean and standard deviation
# s = np.random.normal(mu, sigma, 1000)
# s_2 = np.random.uniform(70, 130, 1000)

# count, bins, ignored = plt.hist(s, 30, density=False, color = 'r')
# # plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) *
# #                np.exp( - (bins - mu)**2 / (2 * sigma**2) )
# #          )
# plt.show()

In [ ]:
# raw_docs = ["Here are some very simple basic sentences.",
# "They won't be very interesting, I'm afraid.",
# "The point of these examples is to _learn how basic text cleaning works_ on *very simple* data."]
raw_docs = validName['merchant_name'].to_list()
tokenized_docs = [word_tokenize(doc) for doc in raw_docs]
print(tokenized_docs[2])

In [ ]:
import re
import string
regex = re.compile('[%s]' % re.escape(string.punctuation)) #see documentation here: http://docs.python.org/2/library/string.html

tokenized_docs_no_punctuation = []

for review in tokenized_docs:
    new_review = []
    for token in review:
        new_token = regex.sub(u'', token)
        if not new_token == u'':
            new_review.append(new_token)
    
    tokenized_docs_no_punctuation.append(new_review)
    
print(tokenized_docs_no_punctuation[2])

In [ ]:
from nltk.corpus import stopwords

tokenized_docs_no_stopwords = []

for doc in tokenized_docs_no_punctuation:
    new_term_vector = []
    for word in doc:
        if not word in stopwords.words('english'):
            new_term_vector.append(word)
    
    tokenized_docs_no_stopwords.append(new_term_vector)

print(tokenized_docs_no_stopwords[2])

In [ ]:
cleaned_sentences = [' '.join(words) for words in tokenized_docs_no_stopwords]
len(cleaned_sentences) == len(validName)

In [ ]:
validName['merchant_name'] = cleaned_sentences

In [ ]:
def shuffle_dataset(dataset):
    """
    Shuffle samples in dataset.

    Args:
        dataset: (list) list of tuples like: [(transaction, start_idx, end_idx), ..... , ]

    Returns:
        dataset: (list) list of tuples like: [,....., (transaction, start_idx, end_idx), ..... , ]

    """

    shuffled_indices = list(range(len(dataset)))
    random.shuffle(shuffled_indices)
    dataset = [dataset[index] for index in shuffled_indices]

    return dataset
def get_merchant_indices_in_sentence(sentence, merchant):
    """
    Given transaction string and merchant string, returns start end end indices of merchant in transaction string.

    Args:
        sentence: (string) transaction string. (example: )
        merchant: (string) merchant name. (example: )

    Returns:
        sentence: (string) converted to upper strings
        start: (int) merchant string start index
        end: (int) merchant string end index

    Examples:
        sentences, start, end = get_sentence_indices("Target 00014423 WATERTOWN MA","target")

        sentences: "TARGET 00014423 WATERTOWN MA"
        start: 0
        end: 6
    """

    sentence = sentence.upper()
    merchant = merchant.upper()

    start = -1
    end = -1

    idx = sentence.find(merchant)
    if idx != -1:
        start = idx
        end = start + len(merchant)

    return sentence, start, end

In [ ]:
validName.sample(10)

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

for idx, row in validName.iterrows():
    # print('Processing row: ', idx)
    # print('row0: ', row[0])
    # print('row1: ', row[1])
    # print(row[0].find(row[1].split(' ')[0]))
    # print(len(row[1]))
    find_i = row[0].find(row[1].split(' ')[0])
    if find_i + len(row[1]) > len(row[0]):   
        row[0] = row[0][:find_i] + row[1]
    
    sentence, start, end = get_merchant_indices_in_sentence(row[0], row[1])
    validName.loc[idx, 'start_idx'] = start
    validName.loc[idx, 'end_idx'] = end

print(validName.sample(50))
#validName.to_csv('data/dataset.csv')

In [ ]:
validName.to_csv('data/dataset.csv')

In [ ]:
cleandataset = cleandataset.drop(columns = 'Unnamed: 0')
cleandataset.columns

In [ ]:
cleandataset.index

In [ ]:
cleandataset = cleandataset[cleandataset['start_idx']!=-1]
len(cleandataset)

In [ ]:
cleandataset.reset_index(drop=True)

In [ ]:
cleandataset.to_csv('data/dataset.csv')

In [ ]:
def character_embedding(text, max_len=300, emb_dim=8):
        """
        Embeds character string with the use of (emb_dim)-bit binary values of each character.

        Args:
            text: (string) text to embed
            max_len: (int) maximum length of text that will be encoded. Padding will be done with zeros.
            emb_dim:

        Returns:
            str_array: (ndarray) 2 dimensional numpy array containing embedded text of shape emb_dim*max_len

        """

        # cut long text with maximum accepted length
        if len(text) > max_len:
            text = text[:max_len]

        str_array = np.zeros((emb_dim, max(len(text), max_len)), dtype=np.int32).tolist()

        for index, char in enumerate(text):
            str_binary = format(ord(char), 'b').zfill(emb_dim)[::-1]
            str_binary = str_binary[:emb_dim]
            for str_index, str_char in enumerate(str_binary, 0):
                str_array[str_index][index] = int(str_char)
       
        padding_str_binary = '0' * emb_dim
        
        for index in range(len(text), max_len):
            for str_index, str_char in enumerate(padding_str_binary, 0):
                str_array[str_index][index] = int(str_char)

        return str_array

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
emb = character_embedding('Martin')
emb

In [ ]:
df = cleandataset.sample(frac=1).reset_index(drop=True)

In [ ]:
train_dataset = df[:int(0.7 * len(df))]
val_dataset = df[int(0.7 * len(df)): int(0.85 * len(df))]
test_dataset = df[int(0.85 * len(df)):]

In [ ]:
train_dataset.head(50)

#### Rule matching

In [ ]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
pattern = [{"LOWER": "hello"}, {"IS_PUNCT": True}, {"LOWER": "world"}]
matcher.add("sID", [pattern])

doc = nlp("Hello, world! Hello world!")
matches = matcher(doc)

for match_id, start, end in matches:
    string_id = nlp.vocab.strings[match_id]  # Get string representation
    span = doc[start:end]  # The matched span
    print(match_id, string_id, start, end, span.text)

In [ ]:
import spacy
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import re

nlp = spacy.load("en_core_web_sm")
doc = nlp("dollar general w main a")
print(doc.ents)
# for idx, row in sample_10.iterrows():
#     # print('processing row: ', idx)
#     # print(row[0])
#     doc = nlp(row[0])
#     all_entities = doc.ents
#     print(all_entities)
#     # print(all_entities)
#     #extraction_temp = str(process.extractOne(row[1], all_entities)[0])
#     #extraction =  re.sub(r"inc|mktp|\d+", "", extraction_temp) 
#     #print('extraction result: ', extraction_temp)

In [ ]:
import pandas as pd
import numpy as np
cleanData = pd.read_csv('data/cleanedmerchant_training.csv')
cleanData.columns

In [ ]:
# stopwordFreeds[(stopwordFreeds['cleanName2'] != stopwordFreeds['CleanName3']) & (stopwordFreeds['CleanName3'] != 'chick fil')].sample(1000)

In [ ]:
cleanData.head()